# 勾配消失問題
多層パーセプトロンでは、層の長さを長くすればするほど表現力は増します。一方で、学習が難しくなるという問題が知られています。

In [ ]:
# Tensorflowが使うCPUの数を制限します。(VMを使う場合)
%env OMP_NUM_THREADS=1
%env TF_NUM_INTEROP_THREADS=1
%env TF_NUM_INTRAOP_THREADS=1

from tensorflow.config import threading
num_threads = 1
threading.set_inter_op_parallelism_threads(num_threads)
threading.set_intra_op_parallelism_threads(num_threads)

#ライブラリのインポート
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# このセルはヘルパー関数です。

# i 番目のレイヤーの出力を取得する
def getActivation(model, i, x):
    from tensorflow.keras import Model
    activation = Model(model.input, model.layers[i].output)(x)
    return activation.numpy()


# iLayer 番目のレイヤーにつながる重み(wij)の勾配を取得する
def getGradientParameter(model, i, x, t):
    from tensorflow.nest import flatten
    from tensorflow import constant
    from tensorflow.python.eager import backprop

    weights = model.layers[i].weights[0]  # get only kernel (not bias)
    with backprop.GradientTape() as tape:
        pred = model(x)
        loss = model.compiled_loss(constant(t), pred)

    gradients = tape.gradient(loss, weights)
    return gradients.numpy()


# 左上: ウェイト(wij)の初期値をプロット
def plot_weights(model, x, t):
    iLayers = [0, 3, 6, 10]
    labels = [
        ' 0th layer',
        ' 3th layer',
        ' 6th layer',
        'Last layer',
    ]

    values = [model.weights[i * 2].numpy().flatten() for i in iLayers]  # get only kernel (not bias)
    plt.hist(values, bins=50, stacked=False, density=True, label=labels, histtype='step')
    plt.xlabel('weight')
    plt.ylabel('Probability density')
    plt.legend(loc='upper left', fontsize='x-small')
    plt.show()

# 左下: 各ノードの出力(sigma(ai))をプロット
def plot_nodes(model, x, t):
    iLayers = [0, 3, 6, 10]
    labels = [
        ' 0th layer',
        ' 3th layer',
        ' 6th layer',
        'Last layer',
    ]

    values = [getActivation(model, i, x).flatten() for i in iLayers]
    plt.hist(values, bins=50, stacked=False, density=True, label=labels, histtype='step')
    plt.xlabel('activation')
    plt.ylabel('Probability density')
    plt.legend(loc='upper center', fontsize='x-small')
    plt.show()

# 右上: ウェイト(wij)の微分(dE/dwij)をプロット
def plot_gradients(model, x, t):
    iLayers = [0, 3, 6, 10]
    labels = [
        ' 0th layer',
        ' 3th layer',
        ' 6th layer',
        'Last layer',
    ]

    grads = [np.abs(getGradientParameter(model, i, x, t).flatten()) for i in iLayers]
    grads = [np.log10(x[x > 0]) for x in grads]
    plt.hist(grads, bins=50, stacked=False, density=True, label=labels, histtype='step')
    plt.xlabel('log10(|gradient of weights|)')
    plt.ylabel('Probability density')
    plt.legend(loc='upper left', fontsize='x-small')
    plt.show()



中間層が10層という深い多層パーセプトロンを用いて、モデル中の重みパラメータの大きさ、勾配の大きさを調べてみます。

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.initializers import RandomNormal, RandomUniform
from tensorflow.keras.layers import Dense

# データセットの生成
nSamples = 1000
nFeatures = 50
x = np.random.randn(nSamples, nFeatures)  # 100個の入力変数を持つイベント1000個生成。それぞれの入力変数は正規分布に従う
t = np.random.randint(2, size=nSamples).reshape([nSamples, 1])  # 正解ラベルは0 or 1でランダムに生成

# モデルの定義
activation = 'sigmoid'  # 中間層の各ノードで使う活性化関数
initializer = RandomNormal(mean=0.0, stddev=1.0)  # weight(wij)の初期値。ここでは正規分布に従って初期化する
# initializer = RandomUniform(minval=-1, maxval=1)  # weight(wij)の初期値。ここでは一様分布に従って初期化する

# 中間層が10層の多層パーセプトロン。各レイヤーのノード数は全て50。
model = Sequential([
    Dense(50, activation=activation, kernel_initializer=initializer, input_dim=nFeatures),
    Dense(50, activation=activation, kernel_initializer=initializer),
    Dense(50, activation=activation, kernel_initializer=initializer),
    Dense(50, activation=activation, kernel_initializer=initializer),
    Dense(50, activation=activation, kernel_initializer=initializer),
    Dense(50, activation=activation, kernel_initializer=initializer),
    Dense(50, activation=activation, kernel_initializer=initializer),
    Dense(50, activation=activation, kernel_initializer=initializer),
    Dense(50, activation=activation, kernel_initializer=initializer),
    Dense(50, activation=activation, kernel_initializer=initializer),
    Dense(1, activation='sigmoid', kernel_initializer=initializer)
])
model.compile(loss='binary_crossentropy', optimizer='adam')

# ウェイト(wij)の初期値をプロット
plot_weights(model, x, t)

# 各ノードの出力(sigma(ai))をプロット
plot_nodes(model, x, t)

# ウェイト(wij)の微分(dE/dwij)をプロット
plot_gradients(model, x, t)


左上のプロットはパラメータ($w_{ij}$)の初期値を表しています。指定したとおり、各層で正規分布に従って初期化されています。
左下のプロットは活性化関数の出力($z_i$)を表しています。パラメータ($w_{ij}$)の初期値として正規分布を指定すると、シグモイド関数の出力はそのほとんどが0か1に非常に近い値となっています。シグモイド関数の微分は$\sigma^{'}(x)=\sigma(x)\cdot(1-\sigma(x))$なので、$\sigma(x)$が0や1に近いときは微分値も非常に小さな値となります。
誤差逆伝播の式は
$$
\begin{align}
\delta_{i}^{(k)} &= \sigma^{'}(a_i^{(k)}) \left( \sum_j w_{ij}^{(k+1)} \cdot \delta_{j}^{(k+1)} \right) \\
\frac{\partial E_n}{\partial w_{ij}^{(k)}}  &= \delta_{j}^{(k)} \cdot z_{i}^{(k)}
\end{align}
$$
でした。$\sigma^{'}(a_i^{(k)})$が小さいと後方の層から前方の層に誤差が伝わる際に、値が小さくなってしまいます。
中上、中下のプロットはそれぞれ各層での$\frac{\partial E_n}{\partial w_{ij}^{(k)}}$、$\delta_{i}^{(k)}$を表しています。
前方の層(0th layer)は後方の層と比較して分布の絶対値が小さくなっています。
右上と右下のプロットは各レイヤーでの$\frac{\partial E_n}{\partial w_{ij}^{(k)}}$と$\delta_{i}^{(k)}$の分布の分散を表しています。

このように誤差が前の層にいくにつれて小さくなるため、前の層が後ろの層と比較して学習が進まなくなります。
この問題は勾配消失の問題として知られています。

勾配消失はパラメータの初期値や、活性化関数を変更することによって解決・緩和することがわかっています。
Kerasの
- [初期化のページ](https://keras.io/ja/initializers/)
- [活性化関数のページ](https://keras.io/ja/activations/)

も参考にしながら、この問題の解決を試みてみましょう。

活性化関数・パラメータの初期化方法の変更はそれぞれコード中の"activation"、"initializer"を変更することによって行えます。

例えばパラメータの初期化を(-0.01, +0.01)の一様分布に変更するときは以下のコードのようにすれば良いです。

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.initializers import RandomNormal, RandomUniform
from tensorflow.keras.layers import Dense

# データセットの生成
nSamples = 1000
nFeatures = 50
x = np.random.randn(nSamples, nFeatures)  # 100個の入力変数を持つイベント1000個生成。それぞれの入力変数は正規分布に従う
t = np.random.randint(2, size=nSamples).reshape([nSamples, 1])  # 正解ラベルは0 or 1でランダムに生成

# モデルの定義
activation = 'sigmoid'  # 中間層の各ノードで使う活性化関数
# initializer = RandomNormal(mean=0.0, stddev=1.0)  # weight(wij)の初期値。ここでは正規分布に従って初期化する
initializer = RandomUniform(minval=-0.01, maxval=0.01)  # weight(wij)の初期値。ここでは一様分布に従って初期化する

# 中間層が10層の多層パーセプトロン。各レイヤーのノード数は全て50。
model = Sequential([
    Dense(50, activation=activation, kernel_initializer=initializer, input_dim=nFeatures),
    Dense(50, activation=activation, kernel_initializer=initializer),
    Dense(50, activation=activation, kernel_initializer=initializer),
    Dense(50, activation=activation, kernel_initializer=initializer),
    Dense(50, activation=activation, kernel_initializer=initializer),
    Dense(50, activation=activation, kernel_initializer=initializer),
    Dense(50, activation=activation, kernel_initializer=initializer),
    Dense(50, activation=activation, kernel_initializer=initializer),
    Dense(50, activation=activation, kernel_initializer=initializer),
    Dense(50, activation=activation, kernel_initializer=initializer),
    Dense(1, activation='sigmoid', kernel_initializer=initializer)
])

model.compile(loss='binary_crossentropy', optimizer='adam')

# ウェイト(wij)の初期値をプロット
plot_weights(model, x, t)

# 各ノードの出力(sigma(ai))をプロット
plot_nodes(model, x, t)

# ウェイト(wij)の微分(dE/dwij)をプロット
plot_gradients(model, x, t)


この例では活性化関数の出力が0.5付近に集中しています。
どのノードも同じ出力をしているということはノード数を増やした意味があまりなくなっており、多層パーセプトロンの表現力が十分に活かしきれていないことがわかります。
また、勾配消失も先程の例と比較して大きくなっています。